In [1]:
import numpy as np
import pandas as pd
import re
from keras.preprocessing.text import Tokenizer
from keras.utils.data_utils import pad_sequences
from sklearn.model_selection import train_test_split
from keras.layers import Embedding, Conv1D, MaxPooling1D, Dense, Flatten, LSTM, Bidirectional
from keras.models import Sequential
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# 選擇數據處理模式：'N-R' 或 'R'
mode = 'R'  # 可以選擇 'N-R' 或 'R'

# 載入與預處理數據
def load_semeval_data(file_paths):
    data = []
    labels = []
    for file_path in file_paths:
        with open(file_path, 'r', encoding='utf-8') as file:
            for line in file:
                parts = line.strip().split('\t')
                if len(parts) == 3:
                    data.append(parts[2])
                    labels.append(parts[1])
    return pd.DataFrame({'tweet': data, 'label': labels})

file_paths = [
    './dataset/train/twitter-2013train-A.txt',
    './dataset/train/twitter-2015train-A.txt',
    './dataset/train/twitter-2016train-A.txt',
]

dataset = load_semeval_data(file_paths)

# 預處理推文文本
def preprocess_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r"https?:\/\/\S+", "", tweet)
    tweet = re.sub(r"[^a-zA-Z0-9\s]", "", tweet)
    return tweet

dataset['tweet'] = dataset['tweet'].apply(preprocess_tweet)

# Tokenization 和 Padding 設置
MAX_NB_WORDS = 10000
MAX_SEQUENCE_LENGTH = 40  # 用於 N-R 模式
REGION_SIZE = 10  # 每個區域的長度，用於 R 模式
NUM_REGIONS = 4  # 區域數量，用於 R 模式
EMBEDDING_DIM = 25

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(dataset['tweet'].values)
word_index = tokenizer.word_index

# 根據模式選擇不同的數據處理方法
if mode == 'N-R':
    # Non-Regional 模式，直接將推文轉為固定長度序列
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)
    input_length = MAX_SEQUENCE_LENGTH  # 設置模型的 input_length
elif mode == 'R':
    # Regional 模式，將推文分成多個區域並填充至相同長度
    X = tokenizer.texts_to_sequences(dataset['tweet'].values)
    X_padded = []
    for tweet in X:
        tweet_regions = []
        for i in range(NUM_REGIONS):
            start = i * REGION_SIZE
            end = start + REGION_SIZE
            region = tweet[start:end]
            if len(region) < REGION_SIZE:
                region = region + [0] * (REGION_SIZE - len(region))
            tweet_regions.extend(region)
        X_padded.append(tweet_regions)
    X = np.array(X_padded)
    input_length = NUM_REGIONS * REGION_SIZE  # 設置模型的 input_length
else:
    raise ValueError("Invalid mode. Choose 'N-R' or 'R'.")

# 標籤處理
label_mapping = {'positive': 0, 'negative': 1, 'neutral': 2}
y = dataset['label'].map(label_mapping).values
y = pd.get_dummies(y).values

# 數據分割
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 載入 GloVe 詞嵌入
embeddings_index = {}
with open(r'C:\Users\jk121\Documents\Code\LargeData\glove.twitter.27B.25d.txt', encoding='utf8') as f:
    for line in f:
        values = line.split()
        word = values[0]
        coefs = np.asarray(values[1:], dtype='float32')
        embeddings_index[word] = coefs

embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

# 定義嵌入層，用於後續模型
embedding_layer = Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                            input_length=input_length, trainable=False)

### 建立單一 CNN 模型並加入 Flatten 層

In [ ]:
from keras.layers import Flatten

cnn_model = Sequential()
cnn_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                        input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_model.add(MaxPooling1D(pool_size=3))
cnn_model.add(Flatten())  # 加入 Flatten 層
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 2s 3ms/step - loss: 1.0111 - accuracy: 0.4634 - val_loss: 0.9568 - val_accuracy: 0.5172
Epoch 2/10
405/405 [==============================] - 1s 2ms/step - loss: 0.9306 - accuracy: 0.5284 - val_loss: 0.9129 - val_accuracy: 0.5487
Epoch 3/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8928 - accuracy: 0.5612 - val_loss: 0.8905 - val_accuracy: 0.5697
Epoch 4/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8747 - accuracy: 0.5693 - val_loss: 0.8858 - val_accuracy: 0.5626
Epoch 5/10
405/405 [==============================] - 1s 3ms/step - loss: 0.8589 - accuracy: 0.5846 - val_loss: 0.8770 - val_accuracy: 0.5731
Epoch 6/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8485 - accuracy: 0.5879 - val_loss: 0.8771 - val_accuracy: 0.5703
Epoch 7/10
405/405 [==============================] - 1s 2ms/step - loss: 0.8373 - accuracy: 0.5996 - val_loss: 0.8676 - val_accuracy: 0.5793
Epoch 

### 單一 LSTM 模型 (Single LSTM)

In [ ]:
from keras.layers import LSTM

# 建立單一 LSTM 模型
lstm_model = Sequential()
lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                         input_length=MAX_SEQUENCE_LENGTH, trainable=False))
lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
lstm_model.add(Dense(3, activation='softmax'))
lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 27s 62ms/step - loss: 0.9677 - accuracy: 0.4824 - val_loss: 0.9273 - val_accuracy: 0.5394
Epoch 2/10
405/405 [==============================] - 25s 61ms/step - loss: 0.9087 - accuracy: 0.5490 - val_loss: 0.8695 - val_accuracy: 0.5756
Epoch 3/10
405/405 [==============================] - 25s 62ms/step - loss: 0.8906 - accuracy: 0.5520 - val_loss: 0.8627 - val_accuracy: 0.5691
Epoch 4/10
405/405 [==============================] - 25s 62ms/step - loss: 0.8803 - accuracy: 0.5716 - val_loss: 0.8681 - val_accuracy: 0.5781
Epoch 5/10
405/405 [==============================] - 25s 63ms/step - loss: 0.8741 - accuracy: 0.5739 - val_loss: 0.8344 - val_accuracy: 0.5883
Epoch 6/10
405/405 [==============================] - 25s 62ms/step - loss: 0.8598 - accuracy: 0.5824 - val_loss: 0.8346 - val_accuracy: 0.5910
Epoch 7/10
405/405 [==============================] - 24s 60ms/step - loss: 0.8464 - accuracy: 0.5919 - val_loss: 0.8239 - val_accuracy:

### CNN + LSTM 組合模型 (CNN + LSTM Combined Network)

In [ ]:
# 建立 CNN + LSTM 模型
cnn_lstm_model = Sequential()
cnn_lstm_model.add(Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
                             input_length=MAX_SEQUENCE_LENGTH, trainable=False))
cnn_lstm_model.add(Conv1D(filters=12, kernel_size=3, activation='relu'))
cnn_lstm_model.add(MaxPooling1D(pool_size=3))
cnn_lstm_model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
cnn_lstm_model.add(Dense(3, activation='softmax'))
cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 12s 24ms/step - loss: 0.9664 - accuracy: 0.4964 - val_loss: 0.9008 - val_accuracy: 0.5450
Epoch 2/10
405/405 [==============================] - 9s 22ms/step - loss: 0.9109 - accuracy: 0.5418 - val_loss: 0.8787 - val_accuracy: 0.5524
Epoch 3/10
405/405 [==============================] - 9s 22ms/step - loss: 0.8860 - accuracy: 0.5601 - val_loss: 0.8752 - val_accuracy: 0.5589
Epoch 4/10
405/405 [==============================] - 9s 22ms/step - loss: 0.8760 - accuracy: 0.5637 - val_loss: 0.8569 - val_accuracy: 0.5756
Epoch 5/10
405/405 [==============================] - 10s 24ms/step - loss: 0.8639 - accuracy: 0.5793 - val_loss: 0.8578 - val_accuracy: 0.5765
Epoch 6/10
405/405 [==============================] - 10s 24ms/step - loss: 0.8556 - accuracy: 0.5817 - val_loss: 0.8443 - val_accuracy: 0.5836
Epoch 7/10
405/405 [==============================] - 9s 22ms/step - loss: 0.8460 - accuracy: 0.5911 - val_loss: 0.8575 - val_accuracy: 0.5

### CNN + LSTM 組合模型 + SVM 組合模型

In [5]:
from sklearn.svm import SVC
from sklearn.metrics import classification_report

# CNN 特徵提取模型
cnn_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=12, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=3),
    Flatten()
])

# LSTM 特徵提取模型
lstm_feature_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Flatten()
])

# 提取 CNN 和 LSTM 特徵
cnn_features = cnn_feature_model.predict(X_train)
lstm_features = lstm_feature_model.predict(X_train)
combined_features = np.hstack((cnn_features, lstm_features))

# 使用 SVM 進行分類
svm_classifier = SVC(kernel='linear', probability=True)
svm_classifier.fit(combined_features, np.argmax(y_train, axis=1))

# 評估
cnn_test_features = cnn_feature_model.predict(X_test)
lstm_test_features = lstm_feature_model.predict(X_test)
combined_test_features = np.hstack((cnn_test_features, lstm_test_features))
y_pred = svm_classifier.predict(combined_test_features)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred))

102/102 [==============================] - 1s 13ms/step
              precision    recall  f1-score   support

           0       0.51      0.64      0.57      1372
           1       1.00      0.00      0.00       466
           2       0.51      0.55      0.53      1397

    accuracy                           0.51      3235
   macro avg       0.67      0.40      0.37      3235
weighted avg       0.58      0.51      0.47      3235



### 單一 3 層 CNN 和 LSTM 網絡

In [ ]:
three_layer_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

three_layer_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 8s 12ms/step - loss: 0.9526 - accuracy: 0.5063 - val_loss: 0.8865 - val_accuracy: 0.5543
Epoch 2/10
405/405 [==============================] - 5s 12ms/step - loss: 0.8895 - accuracy: 0.5609 - val_loss: 0.8648 - val_accuracy: 0.5666
Epoch 3/10
405/405 [==============================] - 4s 11ms/step - loss: 0.8536 - accuracy: 0.5903 - val_loss: 0.8535 - val_accuracy: 0.5833
Epoch 4/10
405/405 [==============================] - 4s 10ms/step - loss: 0.8247 - accuracy: 0.6100 - val_loss: 0.8452 - val_accuracy: 0.5988
Epoch 5/10
405/405 [==============================] - 4s 10ms/step - loss: 0.7924 - accuracy: 0.6311 - val_loss: 0.8536 - val_accuracy: 0.5849
Epoch 6/10
405/405 [==============================] - 4s 10ms/step - loss: 0.7554 - accuracy: 0.6575 - val_loss: 0.8817 - val_accuracy: 0.5861
Epoch 7/10
405/405 [==============================] - 4s 10ms/step - loss: 0.7160 - accuracy: 0.6783 - val_loss: 0.8702 - val_accuracy: 0.5805

### 多重 CNN 和 LSTM 網絡

In [ ]:
multi_cnn_lstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    LSTM(100, dropout=0.2, recurrent_dropout=0.2),
    Dense(3, activation='softmax')
])

multi_cnn_lstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_lstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_lstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 11s 21ms/step - loss: 0.9411 - accuracy: 0.5104 - val_loss: 0.8767 - val_accuracy: 0.5592
Epoch 2/10
405/405 [==============================] - 7s 18ms/step - loss: 0.8831 - accuracy: 0.5611 - val_loss: 0.8521 - val_accuracy: 0.5821
Epoch 3/10
405/405 [==============================] - 8s 19ms/step - loss: 0.8489 - accuracy: 0.5921 - val_loss: 0.8440 - val_accuracy: 0.5805
Epoch 4/10
405/405 [==============================] - 7s 18ms/step - loss: 0.8260 - accuracy: 0.6073 - val_loss: 0.8204 - val_accuracy: 0.6025
Epoch 5/10
405/405 [==============================] - 7s 18ms/step - loss: 0.7942 - accuracy: 0.6335 - val_loss: 0.8339 - val_accuracy: 0.5923
Epoch 6/10
405/405 [==============================] - 8s 19ms/step - loss: 0.7731 - accuracy: 0.6430 - val_loss: 0.8192 - val_accuracy: 0.6006
Epoch 7/10
405/405 [==============================] - 7s 19ms/step - loss: 0.7424 - accuracy: 0.6652 - val_loss: 0.8307 - val_accuracy: 0.604

### 單一 3 層 CNN 和雙向 LSTM 網絡

In [ ]:
from keras.layers import Bidirectional

three_layer_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=32, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=128, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

three_layer_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
three_layer_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = three_layer_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 12s 18ms/step - loss: 0.9552 - accuracy: 0.5045 - val_loss: 0.8864 - val_accuracy: 0.5632
Epoch 2/10
405/405 [==============================] - 8s 19ms/step - loss: 0.8879 - accuracy: 0.5662 - val_loss: 0.8578 - val_accuracy: 0.5901
Epoch 3/10
405/405 [==============================] - 8s 19ms/step - loss: 0.8531 - accuracy: 0.5868 - val_loss: 0.8509 - val_accuracy: 0.5917
Epoch 4/10
405/405 [==============================] - 8s 19ms/step - loss: 0.8252 - accuracy: 0.6112 - val_loss: 0.8393 - val_accuracy: 0.5907
Epoch 5/10
405/405 [==============================] - 7s 18ms/step - loss: 0.7956 - accuracy: 0.6251 - val_loss: 0.8421 - val_accuracy: 0.5898
Epoch 6/10
405/405 [==============================] - 7s 18ms/step - loss: 0.7594 - accuracy: 0.6486 - val_loss: 0.8450 - val_accuracy: 0.5935
Epoch 7/10
405/405 [==============================] - 7s 18ms/step - loss: 0.7228 - accuracy: 0.6649 - val_loss: 0.8837 - val_accuracy: 0.604

### 多重 CNN 和雙向 LSTM 網絡

In [ ]:
multi_cnn_bilstm_model = Sequential([
    Embedding(len(word_index) + 1, EMBEDDING_DIM, weights=[embedding_matrix],
              input_length=MAX_SEQUENCE_LENGTH, trainable=False),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Conv1D(filters=64, kernel_size=3, activation='relu'),
    MaxPooling1D(pool_size=2),
    Bidirectional(LSTM(100, dropout=0.2, recurrent_dropout=0.2)),
    Dense(3, activation='softmax')
])

multi_cnn_bilstm_model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

# 訓練與評估
multi_cnn_bilstm_model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))
y_pred = multi_cnn_bilstm_model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)
print(classification_report(y_true, y_pred_classes))

Epoch 1/10
405/405 [==============================] - 19s 36ms/step - loss: 0.9343 - accuracy: 0.5282 - val_loss: 0.9026 - val_accuracy: 0.5437
Epoch 2/10
405/405 [==============================] - 15s 36ms/step - loss: 0.8631 - accuracy: 0.5784 - val_loss: 0.8542 - val_accuracy: 0.5864
Epoch 3/10
405/405 [==============================] - 14s 35ms/step - loss: 0.8335 - accuracy: 0.6013 - val_loss: 0.8446 - val_accuracy: 0.6000
Epoch 4/10
405/405 [==============================] - 14s 35ms/step - loss: 0.8017 - accuracy: 0.6235 - val_loss: 0.8222 - val_accuracy: 0.6062
Epoch 5/10
405/405 [==============================] - 14s 34ms/step - loss: 0.7735 - accuracy: 0.6394 - val_loss: 0.8163 - val_accuracy: 0.6080
Epoch 6/10
405/405 [==============================] - 14s 34ms/step - loss: 0.7426 - accuracy: 0.6613 - val_loss: 0.8190 - val_accuracy: 0.6201
Epoch 7/10
405/405 [==============================] - 14s 36ms/step - loss: 0.7110 - accuracy: 0.6775 - val_loss: 0.8308 - val_accuracy: